### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anorexia_Nervosa/GSE60190'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

# Check variable availability and data type conversion
# Define keys and functions based on the sample characteristics
import numpy as np

# 'Anorexia_Nervosa' information check
if 'dx: ED' in Sample_Characteristics_Dict[3]:
    trait_row = 3

# 'age' information check    
if all('age:' in item for item in Sample_Characteristics_Dict[5]):
    age_row = 5
    
# 'gender' information check
if 'Sex: F' in Sample_Characteristics_Dict[7] or 'Sex: M' in Sample_Characteristics_Dict[7]:
    gender_row = 7

# Define conversion functions
def convert_trait(value):
    try:
        _, val = value.split(': ')
        if val == 'ED':
            return 1
        elif val == 'Control':
            return 0
    except:
        return None

def convert_age(value):
    try:
        _, val = value.split(': ')
        return float(val)
    except:
        return None

def convert_gender(value):
    try:
        _, val = value.split(': ')
        if val == 'F':
            return 0
        elif val == 'M':
            return 1
    except:
        return None

# Save cohort information
save_cohort_info('GSE60190', './preprocessed/Anorexia_Nervosa/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Anorexia_Nervosa', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Anorexia_Nervosa/trait_data/GSE60190.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
